In [6]:
import pandas as pd
import numpy as np

df = pd.read_excel('ks-projects-201612.xlsx')
df.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,duration
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 11:36:00,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,58.974676
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 00:20:50,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,45.000000
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 04:24:11,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,30.041667
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29 01:00:00,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,55.683993
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000.0,2016-02-26 13:38:27,52375.0,successful,224,US,52375.0,35.000000


In [7]:
#keep only failed and successful
df = df.drop(df[df['state'] == 'canceled'].index)
df = df.drop(df[df['state'] == 'live'].index)
df = df.drop(df[df['state'] == 'undefined'].index)
df = df.drop(df[df['state'] == 'suspended'].index)

#projects longer than 60 days are no longer possible
df = df.drop(df[df['duration']>60].index)

#project ID number not important
df.drop('ID', axis=1, inplace=True)

#want to focus on playing cards
df = df.drop(df[df['category']!='Playing Cards'].index)

In [50]:
df.loc[df['state']=='failed','funded']=0
df.loc[df['state']=='successful','funded']=1

In [51]:
df.loc[df['country']!='US', 'USA']=0
df.loc[df['country']=='US', 'USA']=1

df.loc[df['currency']!='USD', 'currencyType']=0
df.loc[df['currency']=='USD', 'currencyType']=1

In [52]:
df.loc[df['pledged']<df['goal'], 'goalSuccess']=0
df.loc[df['pledged']==df['goal'], 'goalSuccess']=1
df.loc[df['pledged']>df['goal'], 'goalSuccess']=2

In [53]:
def duration(x):
    if x<=30:
        return 0
    if 30<x<=60:
        return 1
    
df['durationType'] = df['duration'].apply(duration)

In [64]:
def backers(x):
    if x < 50:
        return 0
    if 50 < x <=100:
        return 1
    if 100 < x <=500:
        return 2
    if 500 < x <=1000:
        return 3
    if 1000 < x <=5000:
        return 4
    if x > 5000:
        return 5

In [65]:
df['backersType'] = df['backers'].apply(backers)

In [66]:
df['avg_donation'] = df['pledged']/df['backers']
#can help determine when to reward people 

In [73]:
def avg_donation(x):
    if x < 50:
        return 0
    if 50 < x <=100:
        return 1
    if 100 < x <=500:
        return 2
    if 500 < x <=1000:
        return 3
    if 1000 < x <=5000:
        return 4
    if x > 5000:
        return 5

df['avg_donationType'] = df['avg_donation'].apply(avg_donation)

In [ ]:
#what else can create column for to check out correlation

In [74]:
df.head()

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,...,usd pledged,duration,funded,USA,currencyType,goalSuccess,durationType,backersType,avg_donation,avg_donationType
143,"Crap Amidst Hilarity, vol. 1",Playing Cards,Games,USD,2016-03-22 00:36:45,7500.0,2016-01-22 01:36:45,942.25,failed,55,...,942.250000,59.958333,0.0,1.0,1.0,0.0,1,1.0,17.131818,0.0
199,Quick Query - Relaunch!,Playing Cards,Games,USD,2014-08-18 19:02:26,750.0,2014-07-19 19:02:26,1207.00,successful,56,...,1207.000000,30.000000,1.0,1.0,1.0,2.0,0,1.0,21.553571,0.0
204,Strain Wars: Playing cards,Playing Cards,Games,USD,2016-07-31 11:18:43,5000.0,2016-07-16 11:18:43,176.00,failed,6,...,100.000000,15.000000,0.0,1.0,1.0,0.0,0,0.0,29.333333,0.0
402,"Monster Mob, the fun card game for all the mon...",Playing Cards,Games,GBP,2016-07-21 22:29:45,5000.0,2016-06-21 22:29:45,80.00,failed,6,...,29.167912,30.000000,0.0,0.0,0.0,0.0,0,0.0,13.333333,0.0
739,Black Mint Playing Cards,Playing Cards,Games,USD,2016-03-02 17:42:32,10000.0,2016-02-01 17:42:32,125808.40,successful,2140,...,125808.400000,30.000000,1.0,1.0,1.0,2.0,0,4.0,58.788972,1.0


In [75]:
df2 = df.filter(['category','funded','duration','pledged','goal','currencyType','avg_donation','avg_donationType','USA','goalSuccess','durationType','backers','backersType'], axis=1)
df2.head()

,category,funded,duration,pledged,goal,currencyType,avg_donation,avg_donationType,USA,goalSuccess,durationType,backers,backersType
143,Playing Cards,0.0,59.958333,942.25,7500.0,1.0,17.131818,0.0,1.0,0.0,1,55,1.0
199,Playing Cards,1.0,30.000000,1207.00,750.0,1.0,21.553571,0.0,1.0,2.0,0,56,1.0
204,Playing Cards,0.0,15.000000,176.00,5000.0,1.0,29.333333,0.0,1.0,0.0,0,6,0.0
402,Playing Cards,0.0,30.000000,80.00,5000.0,0.0,13.333333,0.0,0.0,0.0,0,6,0.0
739,Playing Cards,1.0,30.000000,125808.40,10000.0,1.0,58.788972,1.0,1.0,2.0,0,2140,4.0


In [76]:
df2.corr()

,funded,duration,pledged,goal,currencyType,avg_donation,avg_donationType,USA,goalSuccess,durationType,backers,backersType
funded,1.000000,-0.201551,0.279840,-0.101938,0.088064,0.100187,0.149518,0.088064,0.998170,-0.049021,0.419070,0.654593
duration,-0.201551,1.000000,-0.030575,0.086343,-0.055856,0.043204,0.037723,-0.055856,-0.200535,0.616638,-0.029986,-0.103220
pledged,0.279840,-0.030575,1.000000,0.280110,-0.054346,0.478848,0.328147,-0.054346,0.281411,-0.006666,0.584161,0.466643
goal,-0.101938,0.086343,0.280110,1.000000,-0.039980,0.387832,0.213816,-0.039980,-0.100808,0.039912,0.049768,0.015945
currencyType,0.088064,-0.055856,-0.054346,-0.039980,1.000000,-0.105539,-0.043209,1.000000,0.087625,-0.042156,0.074171,0.072356
avg_donation,0.100187,0.043204,0.478848,0.387832,-0.105539,1.000000,0.714249,-0.105539,0.098677,0.060986,0.055965,0.064621
avg_donationType,0.149518,0.037723,0.328147,0.213816,-0.043209,0.714249,1.000000,-0.043209,0.147470,0.050398,0.058553,0.064966
USA,0.088064,-0.055856,-0.054346,-0.039980,1.000000,-0.105539,-0.043209,1.000000,0.087625,-0.042156,0.074171,0.072356
goalSuccess,0.998170,-0.200535,0.281411,-0.100808,0.087625,0.098677,0.147470,0.087625,1.000000,-0.046442,0.421636,0.659415
durationType,-0.049021,0.616638,-0.006666,0.039912,-0.042156,0.060986,0.050398,-0.042156,-0.046442,1.000000,0.038956,0.025708


In [77]:
import statsmodels.formula.api as sm
result = sm.ols(formula='funded ~ backersType +avg_donationType + duration -1', data=df2).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 funded   R-squared:                       0.691
Model:                            OLS   Adj. R-squared:                  0.690
Method:                 Least Squares   F-statistic:                     976.7
Date:                Thu, 08 Mar 2018   Prob (F-statistic):               0.00
Time:                        03:12:12   Log-Likelihood:                -614.33
No. Observations:                1314   AIC:                             1235.
Df Residuals:                    1311   BIC:                             1250.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
backersType          0.3097      0.009     34.625      0.000       0.292       0.327
avg_donationType     0.1108      0.018      6.159      0.000       0.075       0.146
duration             0.0027      0.000      5.930      0.000       0.002       0.004
==============================================================================
Omnibus:                       11.216   Durbin-Watson:                   2.048
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               11.245
Skew:                           0.223   Prob(JB):                      0.00362
Kurtosis:                       3.085   Cond. No.                         54.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [84]:
#naive bayes based on correlation and regression work that did, other regressions to try?

def prob_a_b(a,b):
    return a/b

def prob_successful_threeBest(x,y,z):
    return x*y*z

#backersType
backersType0_funded = len(df[(df['funded']==1) & (df['backersType']==0)])
backersType1_funded = len(df[(df['funded']==1) & (df['backersType']==1)])
backersType2_funded = len(df[(df['funded']==1) & (df['backersType']==2)])
backersType3_funded = len(df[(df['funded']==1) & (df['backersType']==3)])
backersType4_funded = len(df[(df['funded']==1) & (df['backersType']==4)])
backersType5_funded = len(df[(df['funded']==1) & (df['backersType']==5)])

#duration
duration30_funded = len(df[(df['funded']==1) & (df['durationType']==0)])
duration60_funded = len(df[(df['funded']==1) & (df['durationType']==1)])

#avg_donation
avg_donation0_funded = len(df[(df['funded']==1) & (df['avg_donationType']==0)])
avg_donation1_funded = len(df[(df['funded']==1) & (df['avg_donationType']==1)])
avg_donation2_funded = len(df[(df['funded']==1) & (df['avg_donationType']==2)])
avg_donation3_funded = len(df[(df['funded']==1) & (df['avg_donationType']==3)])
avg_donation4_funded = len(df[(df['funded']==1) & (df['avg_donationType']==4)])
avg_donation5_funded = len(df[(df['funded']==1) & (df['avg_donationType']==5)])

total_funded = len(df[df['funded']==1])

In [90]:
print(prob_successful_threeBest(prob_a_b(duration30_funded,total_funded),prob_a_b(backersType2_funded,total_funded),\
      prob_a_b(avg_donation0_funded,total_funded)))

#only gives 24% chance, not so great

0.24275748016384405


In [87]:
#greater chance of being funded in 30 days or less
print(prob_a_b(duration30_funded,total_funded))
print(prob_a_b(duration60_funded,total_funded))

0.6766091051805337
0.32339089481946626


In [88]:
#more funding with average donation less than or equal to $50
print(prob_a_b(avg_donation0_funded,total_funded))
print(prob_a_b(avg_donation1_funded,total_funded))
print(prob_a_b(avg_donation2_funded,total_funded))
print(prob_a_b(avg_donation3_funded,total_funded))
print(prob_a_b(avg_donation4_funded,total_funded))
print(prob_a_b(avg_donation5_funded,total_funded))

0.6781789638932496
0.25117739403453687
0.06436420722135008
0.004709576138147566
0.0
0.0


In [89]:
#go for greater than 100 but less than or equal to 500
print(prob_a_b(backersType0_funded,total_funded))
print(prob_a_b(backersType1_funded,total_funded))
print(prob_a_b(backersType2_funded,total_funded))
print(prob_a_b(backersType3_funded,total_funded))
print(prob_a_b(backersType4_funded,total_funded))
print(prob_a_b(backersType5_funded,total_funded))

0.1271585557299843
0.152276295133438
0.5290423861852434
0.13500784929356358
0.05180533751962323
0.0015698587127158557
